# Library

In [1]:
pip install selectivesearch

  Preparing metadata (setup.py) ... done
  Created wheel for selectivesearch: filename=selectivesearch-0.4-py3-none-any.whl size=4335 sha256=ed72ee750317e59d83e2c6af84e23752b71a3c26da4952f7c911055f4fc021aa
  Stored in directory: /root/.cache/pip/wheels/0e/49/95/01447a4e0f48a135ac91fbdb1dd2a1c0523e40e29957b383a3
Successfully built selectivesearch
Note: you may need to restart the kernel to use updated packages.


In [41]:
# Librerie standard
import os
import random
import time
import re
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torchvision.transforms import functional as TF

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from sklearn.svm import SVC

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import shutil 

# Librerie per il parallelismo e il multiprocessing
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

# Librerie per l'ottimizzazione e la gestione delle dipendenze
import selectivesearch

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

# Librerie per la gestione dei dataset
from torch.utils.data import Dataset, DataLoader

# Librerie per modelli e trasformazioni in PyTorch
from torchvision import transforms

# Path

In [7]:
#Output folders and file names
OUT_COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

coco_json_pth = in_dataset_pth / OUT_COCO_JSON_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

# PROPOSALS
OUT_PROPOSALS_FLDR_NM = 'proposals'
prop_fldr = Path(f'/kaggle/working/{OUT_PROPOSALS_FLDR_NM}')
PROP_COCO_JSON_NM = 'proposals.json'
proposals_json = out_dataset_pth / PROP_COCO_JSON_NM
ACTPROP_COCO_JSON_NM ='active_regions.json'
actproposals_json = out_dataset_pth / ACTPROP_COCO_JSON_NM

random.seed(RANDOM_SEED)

In [9]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)
clean_output(prop_fldr)

Cartella /kaggle/working pulita.
Cartella /kaggle/working/proposals non trovata. Nessuna azione necessaria.


In [18]:
import warnings

# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# DataLoader

## Region Proposals Generation

In [10]:
# Funzione per elaborare una singola immagine
def process_single_image(image_data, img_fldr):
    img_id = image_data['id']
    img_name = image_data['file_name']
    img_path = os.path.join(img_fldr, img_name)

    if not os.path.exists(img_path):
        raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

    # Carica l'immagine usando opencv (in modalità RGB)
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converti in RGB
    original_height, original_width, _ = image.shape

    # Ridimensiona l'immagine per velocizzare la Selective Search
    resized_image = cv2.resize(image, (original_width // 2, original_height // 2), interpolation=cv2.INTER_AREA)

    # Genera le region proposals sulla versione ridotta
    processed_proposals = generate_and_process_proposals(resized_image, original_width // 2, original_height // 2)

    # Riscalare le coordinate delle proposte alla dimensione originale
    scaled_proposals = [[x * 2, y * 2, x_max * 2, y_max * 2] for x, y, x_max, y_max in processed_proposals]

    image_data = {
        "image_id": img_id,
        "file_name": img_name,
        "original_size": [original_width, original_height],
        "proposals": []
    }

    for i, proposal in enumerate(scaled_proposals):
        x_min, y_min, x_max, y_max = proposal
        image_data["proposals"].append({
            "proposal_id": i,
            "coordinates": [x_min, y_min, x_max, y_max]
        })

    return image_data

In [11]:
# Funzione per generare le region proposals con Selective Search
def generate_and_process_proposals(image, img_width, img_height):
    img_np = np.array(image, dtype=np.uint8)

    # Esegui la selective search con parametri ottimizzati
    _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.8, min_size=20)

    if len(regions) == 0:
        print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

    processed_proposals = []

    # Pre-filtraggio delle regioni
    for region in regions:
        x, y, w, h = region['rect']
        area = w * h
        if w >= 10 and h >= 10 and 10 <= area <= 0.8 * (img_width * img_height):
            x_max, y_max = x + w, y + h
            processed_proposals.append([x, y, x_max, y_max])

    return processed_proposals

In [12]:
# Funzione per gestire i batch
def batch(iterable, n=1):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, n))
        if not chunk:
            break
        yield chunk

In [13]:
def generate_dataset_proposals(coco_json, img_fldr, output_dir, output_json):
    os.makedirs(output_dir, exist_ok=True)
    all_image_data = []

    # Carica il file JSON di COCO
    with open(coco_json, 'r') as f:
        coco_data = json.load(f)

    # Prepara il mapping delle annotazioni per le immagini
    image_annotations_map = {}
    for annotation in coco_data['annotations']:
        image_id = annotation['image_id']
        if image_id not in image_annotations_map:
            image_annotations_map[image_id] = []
        image_annotations_map[image_id].append(annotation)

    images_with_annotations = [
        image_data for image_data in coco_data['images']
        if image_data['id'] in image_annotations_map and len(image_annotations_map[image_data['id']]) > 0
    ]

    # Parametri per parallelizzazione e batch processing
    max_workers = os.cpu_count() - 1
    batch_size = 500
    total_batches = len(images_with_annotations) // batch_size + (len(images_with_annotations) % batch_size > 0)

    # Processa le immagini in batch con tqdm per monitorare il progresso dei batch
    with tqdm(total=total_batches, desc="Processing batches") as pbar:
        for image_batch in batch(images_with_annotations, batch_size):
            with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
                results = list(executor.map(process_single_image, image_batch, [img_fldr] * len(image_batch)))
            all_image_data.extend(results)
            pbar.update(1)  # Aggiorna la barra di progresso per ogni batch completato

    # Salva il risultato in formato JSON usando orjson
    with open(output_json, 'wb') as json_file:
        json_file.write(orjson.dumps(all_image_data, option=orjson.OPT_INDENT_2))

    print(f"Creato file JSON con le region proposals: {output_json}")

In [ ]:
'''
def generate_dataset_proposals(txt_file, dir_name):
  # prendo i path delle immagini e li memorizzo in una lista
   with open(txt_file, 'r') as f:
            image_paths = [line.strip() for line in f.readlines()]

  img_dir = img_dir
  os.makedirs(img_dir, exist_ok=True)

  for index in range(len(image_paths)):
    img_name = os.path.basename(image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
    img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

    dir_image = os.path.join(dir_name, img_id) #nome della directory che conterrà le region proposals relative all'immagine

    if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
              raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

    img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
    if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
              raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

    image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
    original_width, original_height = image.size

     # GESTIONE DELLE REGION PROPOSALS
     proposals_tensor = generate_region_proposals(image) # image = immagine aperta in formato RGB con la libreria PIL
     #    produce una lista di proposals nel formato (x_min, y_min, x_max, y_max)

     processed_proposals = process_proposals(image_tensor, proposals_tensor) # image_tensor = image dopo la data agumentation
     #    produce le immagine = region proposals relative all'immagine di input

     # salvo le region proposals come immagini in una cartella relativa all'immagine di input
     # - salvataggio in dir_image
     os.makedirs(dir_image, exist_ok=True)

    # Iterare sulle region proposals e salvarle come immagini
    for i, proposal_tensor in enumerate(processed_proposals):
        # Convertire il tensore in immagine PIL (assumendo valori nel range [0, 1])
        proposal_image = Image.fromarray((proposal_tensor.numpy() * 255).astype('uint8'))

        # Generare un nome file unico
        proposal_filename = os.path.join(dir_image, f'proposal_{i:04d}.jpg')

        # Salvare l'immagine
        proposal_image.save(proposal_filename)

        print(f"Salvata proposal {i+1}/{len(processed_proposals)}: {proposal_filename}")

        # Aggiungi il path relativo alla lista
        relative_path = os.path.relpath(proposal_filename, dir_name)  # Path relativo rispetto a dir_name
        all_proposal_paths.append(relative_path)

    # Scrittura di tutti i path relativi in un unico file .txt
    with open(output_txt, 'w') as txt_file:
        for path in all_proposal_paths:
            txt_file.write(f"{path}\n")

    print(f"Creato file TXT con i path relativi di tutte le region proposals: {output_txt}")
'''

In [ ]:
'''
def generate_region_proposals(image, img_width, img_height): #funzione per la generazione delle region proposals per singola immagine
        img_np = np.array(image) #trasformo l'immagine in un array numpy

        if len(img_np.shape) == 3 and img_np.shape[0] == 3: #porto l'immagine nel formato corretto
            img_np = np.transpose(img_np, (1, 2, 0))  # Da [C, H, W] a [H, W, C]
            
        _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.9, min_size=10) #richiamo la funzione di selective search
        #scale: granularità della ricerca (più alto, meno dettagliato) ; 
        #sigma: Standard deviation per il filtro gaussiano usato per la segmentazione ;
        #min_size: Dimensione minima di un segmento nell'algoritmo
        #regions: lista di regioni candidate (proposals).
        # - regione = dizionario che contiene info. -> incluse le coordinate di un rettangolo delimitante (region['rect'])

        #CHECK SULLA PRODUZIONE DELLE REGION PROPOSALS
        if len(regions) == 0:
            print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

        candidate_proposals = []
        for region in regions: #per ogni regione nella lista delle regioni candidate
            x, y, w, h = region['rect'] # prendo le coordinate del rettangolo delimitante
            if w > 0 and h > 0 and w >= 10 and h >= 10: # prendo solo le regioni con altezza e larghezza >= 10 per evitare che siano molto rumorose
                area = w * h
                x_max, y_max = min(x + w, img_np.shape[1]), min(y + h, img_np.shape[0]) # limito la regione alle dimensioni dell'immagine
                candidate_proposals.append([x, y, x_max, y_max, area]) #inserisco la nuova regione nella lista delle region proposals -> aggiungo un valore in più (area) per facilitare il filtraggio dopo

        unique_proposals = list(set(tuple(p) for p in candidate_proposals)) # converto le proposals in tuple in modo da eliminare i duplicati

        #in questo modo non viene preservata la corrispondenza tra region proposals e labels

        #FILTRO LE PROPOSALS PER PRENDERE SOLO QUELLE UTILI/NECESSARIE
        min_area = 10
        max_area_ratio = 0.8
        proposals = []

        for x_min, y_min, x_max, y_max, area in unique_proposals: #per ogni proposal
            if area >= min_area and area <= max_area_ratio * (img_width * img_height):
                proposals.append((x_min, y_min, x_max, y_max))

        return proposals # restituisce le region proposal valide
'''

In [ ]:
'''
def process_proposals(image_tensor, proposals, output_size=(227, 227)):  # la funzione trasforma le proposals trovate in immagini ottenute ritagliando l'immagine originale
    processed_proposals = []
    for proposal in proposals:  # per ogni proposal
        try:
            _, H, W = image_tensor.shape  # vedo le dimensioni dell'immagine
            x_min, y_min, x_max, y_max = map(int, proposal)
            x_min, y_min = max(0, x_min), max(0, y_min)
            x_max, y_max = min(W, x_max), min(H, y_max)

            # Controlla se la proposal ha dimensioni valide per l'immagine di partenza -> tecnicamente non si potrebbe eliminare l'if?
            if x_min < x_max and y_min < y_max:
                cropped_region = image_tensor[:, y_min:y_max, x_min:x_max]  # Ritaglio

                # Controlla che il ritaglio non sia vuoto
                if cropped_region.size == 0:
                    print(f"Ritaglio vuoto per proposal: {proposal}. Salto.")
                    continue

                # Controlla che il tensor sia 3D (C, H, W)
                if cropped_region.ndim != 3:
                    print(f"Proposal non valida per il ridimensionamento: {proposal}. Salto.")
                    continue

                # Converti cropped_region in un tensore PyTorch
                cropped_region = torch.tensor(cropped_region).permute(2, 0, 1)  # Cambia il formato da HWC a CHW

                # Ridimensiona la regione proposta
                resized_region = torch.nn.functional.interpolate(
                    cropped_region.unsqueeze(0), size=output_size, mode='bilinear', align_corners=False
                ).squeeze(0)  # Ridimensiona

                processed_proposals.append(resized_region)
        except Exception as e:
            print(f"Errore durante il processamento della proposal: {proposal}. Errore: {e}")

    return processed_proposals  # Lista di tensori delle region proposals
'''

In [ ]:
'''
def generate_and_process_proposals(image, img_width, img_height):
    img_np = np.array(image)

    # Esegui la selective search per trovare le regioni di interesse (proposals)
    _, regions = selectivesearch.selective_search(img_np, scale=300, sigma=0.9, min_size=10)

    if len(regions) == 0:
        print(f"Warning: Nessuna regione proposta generata per immagine con forma {img_np.shape}.")

    processed_proposals = []  # Lista per le proposte elaborate

    # Pre-filtraggio delle regioni e raccolta delle coordinate delle proposte
    for region in regions:
        x, y, w, h = region['rect']
        
        # Controlla se la regione è abbastanza grande senza calcoli inutili
        area = w * h
        if w >= 10 and h >= 10 and 10 <= area <= 0.8 * (img_width * img_height):
            x_max, y_max = x + w, y + h
            # Aggiungi le coordinate alla lista delle proposte
            processed_proposals.append([x, y, x_max, y_max])

    return processed_proposals  # Restituisce solo le coordinate delle proposte

# Funzione per elaborare un batch di immagini in parallelo
def process_images_in_parallel(images, img_width, img_height):
    # Utilizza concurrent.futures per elaborare più immagini in parallelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # La funzione generate_and_process_proposals verrà applicata su ogni immagine
        results = list(executor.map(lambda image: generate_and_process_proposals(image, img_width, img_height), images))
    return results
'''

In [19]:
generate_dataset_proposals(coco_json_pth, img_fldr, prop_fldr, proposals_json)

Processing batches: 100%|██████████| 65/65 [38:11<00:00, 35.26s/it]


Creato file JSON con le region proposals: /kaggle/working/proposals.json


## Positive Region Proposals

In [37]:
ignored_count = 0  # Contatore globale per le regioni ignorate

def get_iou(bb1, bb2):
    global ignored_count  # Accedi alla variabile globale del contatore

    try:
        # Assicurati che le dimensioni siano corrette
        assert bb1['x1'] < bb1['x2']
        assert bb1['y1'] < bb1['y2']
        assert bb2['x1'] < bb2['x2']
        assert bb2['y1'] < bb2['y2']
    except AssertionError:
        # Se si verifica un errore, incrementa il contatore delle regioni ignorate
        ignored_count += 1
        return 0.0  # Restituisci 0.0 per l'IoU in caso di errore (nessuna sovrapposizione)

    # Calcola le dimensioni dell'area comune tra i due box
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    # Se non c'è sovrapposizione, restituisci 0 come area di intersezione
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # Calcola l'area di intersezione
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    
    # Calcola le aree individuali dei due bounding box
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    
    # Calcola l'area dell'unione
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

    # Verifica che l'IoU sia nel range corretto
    assert iou >= 0.0
    assert iou <= 1.0

    return iou

In [30]:
def assign_and_save_regions(region_json_path, bbox_json_path, image_dir, output_dir, output_json_path, iou_threshold=0.5):
    """Associa le regioni proposte ai bounding boxes, salva le regioni positive come immagini e crea un nuovo JSON con informazioni attivate."""
    
    # Carica i file JSON
    with open(region_json_path, 'r') as f:
        regions = json.load(f)

    with open(bbox_json_path, 'r') as f:
        bboxes = json.load(f)
    
    # Crea un dizionario per cercare annotations per image_id
    annotations_by_image = {}
    for annot in bboxes["annotations"]:
        img_id = annot["image_id"]
        if img_id not in annotations_by_image:
            annotations_by_image[img_id] = []
        # Converte il campo bbox da stringa a lista di float e poi in formato [x1, y1, x2, y2]
        bbox = json.loads(annot["bbox"])  # Converte la stringa in una lista di numeri
        x, y, w, h = bbox
        bbox_converted = [x, y, x + w, y + h]  # Converti in [x1, y1, x2, y2]
        annotations_by_image[img_id].append((torch.tensor(bbox_converted, dtype=torch.float32), annot["category_id"]))
    
    # Crea un dizionario per mappare category_id ai nomi delle categorie
    category_mapping = {cat_id: name for cat_id, name in enumerate(bboxes["categories"])}
    
    # Crea la directory di output se non esiste
    os.makedirs(output_dir, exist_ok=True)

    counter = 0  # Contatore delle immagini salvate
    
    active_region_data = []  # Lista per i dati delle regioni attive

    # Avvolgi il ciclo principale per ogni immagine con tqdm (una barra di progresso generale)
    for image in tqdm(regions, desc="Elaborazione immagini", total=len(regions)):
        image_id = image["image_id"]
        file_name = image["file_name"]
        proposals = image["proposals"]
        
        # Ottieni bounding boxes ground-truth e categorie per l'immagine corrente
        gt_data = annotations_by_image.get(image_id, [])
        if not gt_data:
            # Se non ci sono bounding boxes ground-truth, salta l'immagine
            continue
        
        gt_bboxes = [item[0] for item in gt_data]  # Bounding box ground truth
        gt_categories = [item[1] for item in gt_data]  # Categorie ground truth
        
        # Trasforma proposals in una lista di dizionari compatibili con get_iou
        proposal_coords = [{'x1': p["coordinates"][0], 'y1': p["coordinates"][1], 
                            'x2': p["coordinates"][2], 'y2': p["coordinates"][3]} 
                           for p in proposals]
        
        # Calcola la matrice IoU usando la funzione get_iou
        iou_matrix = []
        for proposal in proposal_coords:
            iou_row = []
            for gt_bbox in gt_bboxes:
                # Ogni gt_bbox deve essere un dizionario simile a proposal
                gt_dict = {'x1': gt_bbox[0].item(), 'y1': gt_bbox[1].item(), 
                           'x2': gt_bbox[2].item(), 'y2': gt_bbox[3].item()}
                iou = get_iou(proposal, gt_dict)
                iou_row.append(iou)
            iou_matrix.append(iou_row)

        # Verifica se la matrice IoU è vuota (ad esempio, se non ci sono bounding box ground-truth per un'immagine, allora gt_bboxes è vuoto)
        if not iou_matrix:
            continue
        
        iou_matrix = torch.tensor(iou_matrix)

        # Identifica le regioni positive (IoU >= soglia)
        max_ious, indices = torch.max(iou_matrix, dim=1)
        positive_indices = torch.nonzero(max_ious >= iou_threshold).squeeze(1)
        
        # Carica l'immagine originale
        image_path = os.path.join(image_dir, file_name)
        original_image = cv2.imread(image_path)
        if original_image is None:
            print(f"Immagine non trovata: {image_path}")
            continue
        
        # Avvolgi il ciclo per ogni proposta positiva senza tqdm (non serve più una barra per ogni proposta)
        for idx in positive_indices:
            x_min, y_min, x_max, y_max = proposal_coords[idx].values()

            # Calcola la larghezza e l'altezza per il formato COCO
            width = x_max - x_min
            height = y_max - y_min
            
            cropped = original_image[int(y_min):int(y_max), int(x_min):int(x_max)]
            
            # Ridimensiona a 224x224
            resized = cv2.resize(cropped, (224, 224), interpolation=cv2.INTER_AREA)
            
            # Ottieni l'etichetta della categoria dal bounding box assegnato
            category_id = gt_categories[indices[idx].item()]
            
            # Salva l'immagine
            output_path = os.path.join(output_dir, f"image_{counter:06d}.jpg")
            cv2.imwrite(output_path, resized)
            
            # Aggiungi la proposta attivata al nuovo JSON in formato COCO
            active_region_data.append({
                "image_id": image_id,
                "file_name": file_name,
                "category_id": category_id,
                "proposal_id": idx.item(),
                "region_bbox": [x_min, y_min, width, height],  
                "saved_path": output_path
            })
            
            counter += 1
    
    # Salva il nuovo JSON con le regioni attive
    with open(output_json_path, 'w') as json_file:
        json.dump(active_region_data, json_file, indent=2)

In [38]:
assign_and_save_regions(proposals_json, coco_json_pth, img_fldr, prop_fldr, actproposals_json, iou_threshold=0.5)

Elaborazione immagini: 100%|██████████| 32199/32199 [11:37<00:00, 46.13it/s] 


In [39]:
print(ignored_count)

412


In [45]:
# controllo sulle regioni
from collections import Counter

file_path = '/kaggle/working/active_regions.json'

#carica il file JSON
with open(file_path, 'r') as f:
    data = json.load(f)

#conta numero di regioni
num_regioni = len(data)
print(f"Numero di regioni: {num_regioni}")

# occorrenze dei category_id
category_ids = [entry['category_id'] for entry in data]
category_counts = Counter(category_ids)
print("Occorrenze dei category_id:", category_counts)


Numero di regioni: 180600
Occorrenze dei category_id: Counter({6: 159041, 1: 6132, 9: 3563, 4: 3488, 2: 2765, 0: 2731, 5: 1226, 8: 1159, 3: 329, 10: 98, 7: 68})


## Custom Dataset

In [ ]:
'''
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, coco_json_file, aug=False):
        def generate_id(file_name): #prende il nome.jpg di una immagine e restituisce solo l'identificativo senza prefissi e suffissi
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')

        with open(txt_file, 'r') as f: #salva le region proposals in un file txt con direttamente le info della cartella
            self.image_paths = [line.strip() for line in f.readlines()] #memorizzo i path delle immagini in una lista

        with open(coco_json_file, 'r') as f: #leggo il file .json - contenente (...) - con coco
            coco_data = json.load(f)

        self.image_annotations = {} #dizionario contenente per ogni immagine una lista di categorie di oggetti presebti

        for annotation in coco_data['annotations']: #uso la sezione annotazioni del file .json per ricavare delle info. sulle immagini del dataset
            image_id = annotation['image_id']
            category_id = annotation['category_id'] # lista di category_id = categorie degli oggetti nell'immagine)

            if image_id not in self.image_annotations: #verifico se l'id dell'immagine è già presente nel dizionario
                self.image_annotations[image_id] = []

            self.image_annotations[image_id].append(category_id)

        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        } #dizionario in cui per ogni nome dell'immagine ottenuta da generate_id(file_name) associa il nome.jpg dell'imagine

        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # trasformazione di base da applicare a tutte le immagini

        # lasciare momentaneamente in caso di aggiornamenti futuri
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) # strasformazione per la data agumentation

        self.aug = aug

    def __len__(self): # ritorna il numero di elementi in self.image_paths -> chi è?
        return len(self.image_paths)

    def __getitem__(self, index):
        img_name = os.path.basename(self.image_paths[index]) #prendo il path dell'immagine da self.image_paths in base all'indice fornito
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', '')) #ricavo l'id dell'immagine -> non l'ho già fatto nell'init?

        if img_id not in self.image_info: #vedo dal dizionario self.image_info se l'immagine è contenuta nel file COCO
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")

        img_path = os.path.join(self.img_dir, img_name) #prendo il path completo dell'immagine unendo il path della cartella con il nome.jpg dell'immagine
        if not os.path.exists(img_path): #se il path non esiste allora lo segnalo
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")

        image = Image.open(img_path).convert('RGB') #apro l'immagine e ne ricavo le dimensioni
        #original_width, original_height = image.size

        if self.aug: #se la variabile self.aug è alta allora applico la self.aug_transform altrimenti la self.base_transform _> HA SENSO? LE DUE FUNZIONI SONO = !!
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)

        #POTREBBE ESSERCI UN PROBLEMA NELLA CORRISPONDENZA TRA LABLES E REGION PROPOSALS -> perchè in proposals_tensor non ci sono tutte le region proposals perchè alcune vengono scartate
        # -> se la corrispondenza è 1 a 1 allora potrebbe convenire dare a _generate_region_proposals(image) sia le labes che l'immagine? -> non è certo perchè
        # in lables c'è una lista di lable ma non viene specificato dove sono localizzate

        # restituisce un dizionario
        return {
            "regions": image_tensor  # region proposals elaborate, una lista di tensori che rappresentano regioni candidate per il rilevamento
        }
'''

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, json_file, transform=None):
        """
        Inizializza il dataset.

        :param json_file: Percorso del file JSON contenente le informazioni sulle regioni.
        :param transform: Trasformazioni da applicare alle immagini. Se non fornito, vengono usate trasformazioni di default.
        """
        with open(json_file, 'r') as f:
            self.data = json.load(f)  # Carica il file JSON
        
        # Trasformazioni di default se non vengono fornite
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),         # Ridimensiona l'immagine a 224x224
            transforms.ToTensor(),                 # Converte l'immagine in un tensore
            transforms.Normalize(                  # Normalizzazione con valori di ImageNet
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225]
            )
        ])  

    def __len__(self):
        """Restituisce il numero totale di immagini/proposte nel dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """Restituisce un esempio (immagine e etichetta) per l'addestramento."""
        # Carica l'esempio dal file JSON
        sample = self.data[idx]
        
        # Carica l'immagine
        image = Image.open(sample["saved_path"]).convert("RGB")
        
        # Etichetta della categoria
        label = sample["category_id"]  # Categoria della proposta

        # Applica le trasformazioni (ad esempio, ridimensionamento, normalizzazione)
        image = self.transform(image)
        
        return image, label

## Feature Extraction

In [ ]:
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        super(AlexNet, self).__init__()
        self._output_num = num_classes

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
     
        self.drop8 = nn.Dropout()
        self.fn8 = nn.Linear(256 * 6 * 6, 4096)
        self.active8 = nn.ReLU(inplace=True)
        
        self.drop9 = nn.Dropout()
        self.fn9 = nn.Linear(4096, 4096)
        self.active9 = nn.ReLU(inplace=True)
        
        self.fn10 = nn.Linear(4096, self._output_num)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.drop8(x)
        x = self.fn8(x)
        x = self.active8(x)

        x = self.drop9(x)
        x = self.fn9(x)
        
        feature = self.active9(x)
        final = func.sigmoid(self.fn10(feature))

        return feature, final